Backpropagation
=====

### Reference : http://www.hanbit.co.kr/store/books/look.php?p_code=B8475831198
--------

- 계산 그래프를 통해 국소적 계산 
- 계산 그래프의 이점  
1) 각 노드의 단순한 계산에 집중하여 문제를 단순화할 수 있다  
2) 역전파를 통해 미분을 효율적으로 할 수 있음(각 노드가 결과에 미치는 영향을 수치적으로 파악할 수 있음)  

### 1. 단순한 계층 구현하기

#### 1.1 곱셈계층

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


#### 1.2 덧셈계층

In [5]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

### 2. 활성화 함수 계층 구하기

#### 2.1 ReLU 계층

In [6]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)  # 값이 0 이하인 노드의 값에 True값을 할당함
        out = x.copy()
        out[self.mask] = 0
        
    def backward(self, dout):
        dout[self.mask] = 0  # True인 값의 역전파 값은 0으로 만듦
        dx = dout
        
        return dx

#### 2.2 Sigmoid 계층

## $y = \frac {1} {1 + exp(-x)}$

### 2.2.1 Sigmoid의순전파
#### 0. input $ x$
#### 1. $-1 * x$ : (-1) 곱해줌
#### 2. $exp(-x)$ : 자연상수를 밑으로 가짐
#### 3. $1 + exp(-x)$ : 1을 더해줌
#### 4. $y = \frac 1 {1 + exp(-x)}$ : 역수를 취함
-------------------------  
    
### 2.2.2 Sigmoid의 역전파
#### 0. input $\frac {\partial L} {\partial y}$ : Loss function에 대한 예측값 y의 편미분값
#### 1. $\frac {\partial L} {\partial y} * \frac {\partial L} {\partial \frac 1 x}$ : Loss function에 대한 역수(1/x)의 편미분 값($-y^2$)
#### 2.$\frac {\partial L} {\partial y} * \frac {\partial L} {\partial \frac 1 x}$ : Loss function에 대한 상수의 편미분 값을 곱해줌(=1)
#### 3. $\frac {\partial L} {\partial y} * \frac {\partial L} {\partial \frac 1 x} * \frac {\partial L} {\partial exp(x)}$ : Loss function에 대한 자연상수(exp(x))의 편미분 값($exp(x)$)
#### 4. $- \frac {\partial L} {\partial y} * \frac {\partial L} {\partial \frac 1 x} * \frac {\partial L} {\partial exp(x)}$ : Loss function에 대한 상수의 편미분 값을 곱해줌(*-1)

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

#### 2.3 Affine/Softmax 계층 구현하기

- 신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서는 어파인 변환(Affine transformation)이라고 함

In [8]:
import numpy as np

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = x
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)  # 전치행렬!!!!
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

#### 2.4 Softmax-with-Loss 계층

In [11]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

### 3. 오차역전파법 구현하기

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
